# Imports

In [1]:
import boto3
import json
import numpy as np
import pandas as pd
import random

# Create a session

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
region = "eu-west-1"

In [3]:
try:
    session = boto3.Session(
        aws_access_key_id=user_secrets.get_secret("access_key"),
        aws_secret_access_key=user_secrets.get_secret("secret_access_key"),
        region_name=region
    )
except Exception as e:
    print("Error occurred while creating session: ", e)

In [4]:
try:
    personalize = session.client('personalize')
    personalize_runtime = session.client('personalize-runtime')
except Exception as e:
    print("Error creating client: ", e)

# Get CVSs

In [8]:
items = pd.read_csv('/kaggle/input/aws-dataset/items.csv')
users = pd.read_csv('/kaggle/input/aws-dataset/users.csv')
interactions = pd.read_csv('/kaggle/input/aws-dataset/interactions.csv')

In [9]:
items.head()

,Unnamed: 0,ITEM_ID,ACTORS,DIRECTOR,GENRES,TITLE,RELEASE_YEAR,eventValue,CREATION_TIMESTAMP
0,0,920962,"Brad Pitt, Logan Lerman, Michael Peña, Shia La...",David Ayer,"War, Drama, Action",Fury,2022,7.268,1640995200
1,1,920961,"Clint Eastwood, Marianne Koch, Gian Maria Volo...",Sergio Leone,Western,A Fistful of Dollars,2017,7.623,1483228800
2,2,920960,"Josh Brolin, Javier Bardem, Tommy Lee Jones, K...","Joel Coen, Ethan Coen","Crime, Drama, Thriller",No Country for Old Men,2023,6.522,1672531200
3,3,920959,"Ben Cross, Ian Charleson, Cheryl Campbell, Ali...",Hugh Hudson,"Drama, History",Chariots of Fire,2011,6.057,1293840000
4,4,920958,"Steve McQueen, James Garner, Richard Attenboro...",John Sturges,"Adventure, Drama, History, Thriller, War",The Great Escape,2023,6.522,1672531200


In [10]:
users.head()

,Unnamed: 0,USER_ID,AGE,SEX
0,0,1,29,M
1,1,2,27,F
2,2,3,31,M
3,3,4,20,M
4,4,5,35,F


In [11]:
interactions.head()

,Unnamed: 0,USER_ID,ITEM_ID,EVENT_TYPE,eventValue,TIMESTAMP
0,0,1,918382,watch,5,1681289127
1,1,1,920916,watch,5,1681289127
2,2,1,918356,watch,5,1681289127
3,3,1,920923,watch,5,1681289127
4,4,1,918307,watch,5,1681289127


# Get Recommendations for a user

In [13]:
recommender_top_picks_arn = input('recommender top-picks arn: ')

recommender top-picks arn:  arn:aws:personalize:eu-west-1:267355461125:recommender/fast-bachelor-project-recommender


In [114]:
test_user_id = int(random.choice(users['USER_ID']))

In [115]:
get_recommendations_response = personalize_runtime.get_recommendations(
    recommenderArn = recommender_top_picks_arn,
    userId = str(test_user_id),
    numResults = 5
)

In [116]:
def get_movie_by_id(movie_id, movie_df):    
    try:
        return movie_df.loc[movie_df["ITEM_ID"] == int(movie_id)].iloc[0]
    except:
        print(movie_id)
        return "Error obtaining movie information"

In [117]:
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    movie = get_movie_by_id(item['itemId'], items)
    recommendation_list.append(movie)

In [118]:
user_recommendations = pd.DataFrame(recommendation_list)
user_recommendations = user_recommendations.drop('Unnamed: 0', axis=1)

In [119]:
user_recommendations.head()

,ITEM_ID,ACTORS,DIRECTOR,GENRES,TITLE,RELEASE_YEAR,eventValue,CREATION_TIMESTAMP
39,920923,"Harrison Ford, Mark Hamill, Carrie Fisher, Ada...",J.J. Abrams,"Adventure, Action, Science Fiction, Fantasy",Star Wars: The Force Awakens,2002,7.965,1009843200
89,920872,"Mel Gibson, Catherine McCormack, Sophie Marcea...",Mel Gibson,"Action, Drama, History, War",Braveheart,2002,7.965,1009843200
197,918354,"Chris Pratt, Zoe Saldaña, Dave Bautista, Vin D...",James Gunn,"Action, Science Fiction, Adventure",Guardians of the Galaxy,2014,7.903,1388534400
99,920862,"Philippe Petit, Jean François Heckel, Jean-Lou...",James Marsh,Documentary,Man on Wire,2014,7.903,1388534400
86,920875,"Alex D. Linz, Olek Krupa, Rya Kihlstedt, Lenny...",Raja Gosnell,"Comedy, Family",Home Alone 3,2001,6.942,978307200


# Get Movies of the interactions from the same user

In [120]:
def get_user_movies(user_id, interactions, movies):  
    user_interactions = interactions.loc[interactions['USER_ID'] == user_id]
    user_movie_ids = user_interactions['ITEM_ID'].tolist()
    user_movies = movies[movies['ITEM_ID'].isin(user_movie_ids)]
    return user_movies

In [121]:
user_movies = get_user_movies(test_user_id, interactions, items)

In [124]:
df = user_movies.drop("Unnamed: 0", axis=1)

In [125]:
df.head()

,ITEM_ID,ACTORS,DIRECTOR,GENRES,TITLE,RELEASE_YEAR,eventValue,CREATION_TIMESTAMP
21,920941,"Paul Newman, Robert Redford, Katharine Ross, S...",George Roy Hill,"Western, Drama, Crime",Butch Cassidy and the Sundance Kid,2010,8.400,1262304000
47,920915,"Alden Ehrenreich, Joonas Suotamo, Woody Harrel...",Ron Howard,"Science Fiction, Adventure, Action",Solo: A Star Wars Story,2002,8.377,1009843200
49,920913,"Keanu Reeves, Carrie-Anne Moss, Yahya Abdul-Ma...",Lana Wachowski,"Science Fiction, Action, Adventure",The Matrix Resurrections,2014,7.903,1388534400
83,920878,"Dwayne Johnson, Kevin Hart, Jack Black, Karen ...",Jake Kasdan,"Adventure, Action, Comedy, Fantasy",Jumanji: Welcome to the Jungle,2014,8.399,1388534400
133,920828,"Johnny Knoxville, Jackson Nicoll, Georgina Cat...",Jeff Tremaine,Comedy,Jackass Presents: Bad Grandpa,2014,8.399,1388534400
